In [8]:
from numpy import *
import numpy as np
import math
import re
import glob
import os
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn import linear_model

In [9]:
os.chdir("/Users/Ruihao/Desktop/course/AML/HW3/data/")
labels = [] 
test_feature = []
test_label = []
train_feature = []
train_label = []
all_files = os.listdir("/Users/Ruihao/Desktop/course/AML/HW3/data")
txt_files = filter(lambda x: x[-4:] == '.txt', all_files)
for filename in txt_files:
    f = open(filename, 'r')
    posCnt = 0; negCnt = 0
    sampThreshold = 800
    for line in f:
        temp = line.split('\t')
        label = temp[1][0]
        labels.append(label) 
        sentence = filter(None, re.split(r'[^a-zA-Z\']', temp[0].lower()))
        stop = set(stopwords.words('english'))
        lmtzr = WordNetLemmatizer()
        parsed = [lmtzr.lemmatize(word) for word in sentence if word not in stop]
        if label == '1' and posCnt < 400:
            train_feature.append(parsed)
            train_label.append(label)
            posCnt += 1
            continue
        if label == '0' and negCnt < 400:
            train_feature.append(parsed)
            train_label.append(label)
            negCnt += 1
            continue
        test_feature.append(parsed)
        test_label.append(label)
#print train_feature[924]
#print train_feature[1574]

In [3]:
def BagOfWordModel(train_feature, test_feature):
    dictionary = list()  
    for i in range(len(train_feature)):
        for j in range(len(train_feature[i])):
            if train_feature[i][j] not in dictionary:
                dictionary.append(train_feature[i][j])
    train_feature_data = np.zeros((len(train_feature), len(dictionary)))  
    for i in range(len(train_feature)):
        for j in range(len(train_feature[i])):
            train_feature_data[i][dictionary.index(train_feature[i][j])] += 1
    test_feature_data = np.zeros((len(test_feature), len(dictionary)))      
    for i in range(len(test_feature)):
        for j in range(len(test_feature[i])):
            if test_feature[i][j] in dictionary: 
                test_feature_data[i][dictionary.index(test_feature[i][j])] += 1
    return train_feature_data, test_feature_data, dictionary

In [4]:
# L2 Norm postprocessing strategy
def postprocessing(train_feature_data):
    for i in range(len(train_feature_data)):
        x = sum([k*k for k in train_feature_data[i]])
        x = math.sqrt(x)
        if x == 0: continue
        for j in range(len(dictionary)):
            train_feature_data[i][j] /= x
    return train_feature_data

In [5]:
import heapq
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB

train_feature_data, test_feature_data, dictionary = BagOfWordModel(train_feature, test_feature)
postprocessing(train_feature_data)
print ("logistic regression based on Bag of Word Model: ")
logistic = linear_model.LogisticRegression()
res = logistic.fit(train_feature_data, train_label)
print res.score(test_feature_data, test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    for j in range(0, len(res.coef_[0])):
        if res.coef_[0][j] == i: print dictionary[j]

print ("***************************************************")
print ("Naive Bayes Classifier based on Bag of Word Model: ")
gnb = GaussianNB()
res = gnb.fit(train_feature_data,train_label)
print res.score(test_feature_data,test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.sigma_[0])
for i in nmax:
    for j in range(0, len(res.sigma_[0])):
        if res.sigma_[0][j] == i: print dictionary[j]

logistic regression based on Bag of Word Model: 
0.793333333333
[[239  61]
 [ 63 237]]
great
love
excellent
nice
delicious
good
best
amazing
loved
fantastic
***************************************************
Naive Bayes Classifier based on Bag of Word Model: 
0.73
[[237  63]
 [ 99 201]]
bad
movie
service
phone
back
time
good
food
don't
film


In [6]:
def NgramModel(train_feature, test_feature):
    dictionary = list()
    for i in range(len(train_feature)):
        for j in range(len(train_feature[i])-1):
            pair = train_feature[i][j]+" "+train_feature[i][j+1]
            if pair not in dictionary:
                dictionary.append(pair)
    train_feature_data = np.zeros((len(train_feature), len(dictionary)))  
    for i in range(len(train_feature)):
        for j in range(len(train_feature[i])-1):
            pair = train_feature[i][j] + " " + train_feature[i][j+1]
            train_feature_data[i][dictionary.index(pair)] += 1
        
    test_feature_data = np.zeros((len(test_feature), len(dictionary)))  
    for i in range(len(test_feature)):
        for j in range(len(test_feature[i])-1):
            pair = test_feature[i][j]+" "+test_feature[i][j+1]
            if pair in dictionary:  
                test_feature_data[i][dictionary.index(pair)] += 1
    return train_feature_data, test_feature_data, dictionary

train_feature_data, test_feature_data, dictionary = NgramModel(train_feature, test_feature)
postprocessing(train_feature_data)
print ("logistic regression based on 2-gram Model: ")
logistic = linear_model.LogisticRegression()
res = logistic.fit(train_feature_data, train_label)
print res.score(test_feature_data, test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    for j in range(0, len(res.coef_[0])):
        if res.coef_[0][j] == i: print dictionary[j]
            
print ("**********************************************")
print ("Naive Bayes Classifier based on 2-gram Model: ")
gnb = GaussianNB()
res = gnb.fit(train_feature_data,train_label)
print res.score(test_feature_data,test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.sigma_[0])
for i in nmax:
    for j in range(0, len(res.sigma_[0])):
        if res.sigma_[0][j] == i: 
            print dictionary[j]

logistic regression based on 2-gram Model: 
0.638333333333
[[266  34]
 [183 117]]
work great
highly recommend
one best
great phone
great product
food good
easy use
great food
really good
good price
**********************************************
Naive Bayes Classifier based on 2-gram Model: 
0.631666666667
[[266  34]
 [187 113]]
waste time
waste money
customer service
don't waste
won't back
don't buy
would recommend
go back
sending back
movie bad


In [7]:
def PCA(train, test, r):
    # subtract mean
    mean_vector = np.mean(train, axis=0)
    X = train - mean_vector
    X = np.dot(X.transpose(), X)
    U, s, V = np.linalg.svd(X, full_matrices = True)
    F = np.dot(train, (V[:r,:]).T)
    K = np.dot(test, (V[:r,:]).T)
    return F, K

# test PCA implement choose r = 10, 50, 100
print "**************************************************"
print "accuracy with r = 100:",
print "logistic regression based on Bag of Word Model: "
train_feature_data, test_feature_data, dictionary = BagOfWordModel(train_feature, test_feature)
postprocessing(train_feature_data)
train_feature_data, test_feature_data = PCA(train_feature_data, test_feature_data, 100)
logistic = linear_model.LogisticRegression()
res = logistic.fit(train_feature_data, train_label)
print res.score(test_feature_data, test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    for j in range(0, len(res.coef_[0])):
        if res.coef_[0][j] == i: print dictionary[j]
print "Naive Bayes Classifier based on Bag of Word Model: "
gnb = GaussianNB()
res = gnb.fit(train_feature_data,train_label)
print res.score(test_feature_data,test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.sigma_[0])
for i in nmax:
    for j in range(0, len(res.sigma_[0])):
        if res.sigma_[0][j] == i: print dictionary[j]
print "logistic regression based on 2-gram Model: "
train_feature_data, test_feature_data, dictionary = NgramModel(train_feature, test_feature)
postprocessing(train_feature_data)
logistic = linear_model.LogisticRegression()
res = logistic.fit(train_feature_data, train_label)
print res.score(test_feature_data, test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    for j in range(0, len(res.coef_[0])):
        if res.coef_[0][j] == i: print dictionary[j]
print ("Naive Bayes Classifier based on 2-gram Model: ")
gnb = GaussianNB()
res = gnb.fit(train_feature_data,train_label)
print res.score(test_feature_data,test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.sigma_[0])
for i in nmax:
    for j in range(0, len(res.sigma_[0])):
        if res.sigma_[0][j] == i: 
            print dictionary[j]           
print "**************************************************"
print "accuracy with r = 50:",
print "logistic regression based on Bag of Word Model: "
train_feature_data, test_feature_data, dictionary = BagOfWordModel(train_feature, test_feature)
postprocessing(train_feature_data)
train_feature_data, test_feature_data = PCA(train_feature_data, test_feature_data, 50)
logistic = linear_model.LogisticRegression()
res = logistic.fit(train_feature_data, train_label)
print res.score(test_feature_data, test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    for j in range(0, len(res.coef_[0])):
        if res.coef_[0][j] == i: print dictionary[j]
print "Naive Bayes Classifier based on Bag of Word Model: "
gnb = GaussianNB()
res = gnb.fit(train_feature_data,train_label)
print res.score(test_feature_data,test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.sigma_[0])
for i in nmax:
    for j in range(0, len(res.sigma_[0])):
        if res.sigma_[0][j] == i: print dictionary[j]
print "logistic regression based on 2-gram Model: "
train_feature_data, test_feature_data, dictionary = NgramModel(train_feature, test_feature)
postprocessing(train_feature_data)
logistic = linear_model.LogisticRegression()
res = logistic.fit(train_feature_data, train_label)
print res.score(test_feature_data, test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    for j in range(0, len(res.coef_[0])):
        if res.coef_[0][j] == i: print dictionary[j]
print ("Naive Bayes Classifier based on 2-gram Model: ")
gnb = GaussianNB()
res = gnb.fit(train_feature_data,train_label)
print res.score(test_feature_data,test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.sigma_[0])
for i in nmax:
    for j in range(0, len(res.sigma_[0])):
        if res.sigma_[0][j] == i: 
            print dictionary[j]
print "**************************************************"
print "accuracy with r = 10:",
print "logistic regression based on Bag of Word Model: "
train_feature_data, test_feature_data, dictionary = BagOfWordModel(train_feature, test_feature)
postprocessing(train_feature_data)
train_feature_data, test_feature_data = PCA(train_feature_data, test_feature_data, 10)
logistic = linear_model.LogisticRegression()
res = logistic.fit(train_feature_data, train_label)
print res.score(test_feature_data, test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    for j in range(0, len(res.coef_[0])):
        if res.coef_[0][j] == i: print dictionary[j]
print "Naive Bayes Classifier based on Bag of Word Model: "
gnb = GaussianNB()
res = gnb.fit(train_feature_data,train_label)
print res.score(test_feature_data,test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.sigma_[0])
for i in nmax:
    for j in range(0, len(res.sigma_[0])):
        if res.sigma_[0][j] == i: print dictionary[j]
print "logistic regression based on 2-gram Model: "
train_feature_data, test_feature_data, dictionary = NgramModel(train_feature, test_feature)
postprocessing(train_feature_data)
logistic = linear_model.LogisticRegression()
res = logistic.fit(train_feature_data, train_label)
print res.score(test_feature_data, test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    for j in range(0, len(res.coef_[0])):
        if res.coef_[0][j] == i: print dictionary[j]
print ("Naive Bayes Classifier based on 2-gram Model: ")
gnb = GaussianNB()
res = gnb.fit(train_feature_data,train_label)
print res.score(test_feature_data,test_label)
print confusion_matrix(test_label, res.predict(test_feature_data))
nmax = heapq.nlargest(10, res.sigma_[0])
for i in nmax:
    for j in range(0, len(res.sigma_[0])):
        if res.sigma_[0][j] == i: 
            print dictionary[j]


**************************************************
accuracy with r = 100: logistic regression based on Bag of Word Model: 
0.738333333333
[[230  70]
 [ 87 213]]
razr
phone
mere
problem
design
ft
right
clip
far
wasted
Naive Bayes Classifier based on Bag of Word Model: 
0.631666666667
[[211  89]
 [132 168]]
unless
go
case
u
value
excellent
converter
great
plug
jawbone
logistic regression based on 2-gram Model: 
0.638333333333
[[266  34]
 [183 117]]
work great
highly recommend
one best
great phone
great product
food good
easy use
great food
really good
good price
Naive Bayes Classifier based on 2-gram Model: 
0.631666666667
[[266  34]
 [187 113]]
waste time
waste money
customer service
don't waste
won't back
don't buy
would recommend
go back
sending back
movie bad
**************************************************
accuracy with r = 50: logistic regression based on Bag of Word Model: 
0.69
[[233  67]
 [119 181]]
razr
problem
right
wasted
fun
waste
value
go
jawbone
quality
Naive Bayes Class